<a href="https://colab.research.google.com/github/danjshaw/ece57000-finalProject/blob/main/BERT_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial Setup

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [1]:
!pip install datasets
!pip install evaluate
!pip freeze > requirements.txt
import torch
import torch.nn as nn
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.4 MB/s eta 0:00:00
cuda


# Full Fine-tuning



Hyperparameters from [google-research/bert](https://github.com/google-research/bert):



> For each task, we selected the best fine-tuning hyperparameters from the lists below, and trained for 4 epochs:
> * batch sizes: 8, 16, 32, 64, 128
> * learning rates: 3e-4, 1e-4, 5e-5, 3e-5



In [4]:
epochs = 4
batch_sizes = [8, 16, 32, 64, 128]
learning_rates = [3e-4, 1e-4, 5e-5, 3e-5]

In [47]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding, set_seed

set_seed(0)
raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "prajjwal1/bert-tiny"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [50]:
from transformers import AutoModelForSequenceClassification

ft_model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
import evaluate

def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [52]:
from transformers import Trainer, TrainingArguments

ft_results = []
runs = len(batch_sizes)*len(learning_rates)
run = 0
for size in batch_sizes:
  for rate in learning_rates:
    run += 1
    print(f"Run: {run}/{runs}")
    ft_result = {"batch_size": size, "learning_rate": rate}

    training_args = TrainingArguments(
        "test-trainer",
        eval_strategy="epoch",
        per_device_eval_batch_size=size,
        per_device_train_batch_size=size,
        num_train_epochs=epochs,
        learning_rate=rate,
        disable_tqdm=True,
        report_to="none"
    )

    trainer = Trainer(
        ft_model,
        training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        data_collator=data_collator,
        processing_class=tokenizer,
        compute_metrics=compute_metrics,
    )

    ft_result = ft_result | trainer.train().metrics | trainer.evaluate()
    ft_results.append(ft_result)

ft_max = ft_results[0]
for i, result in enumerate(ft_results):
  if result['eval_f1'] > ft_max['eval_f1']:
    ft_max = result

print(f'\n\n================\
        \nBest Result: \
        \nbatch_size={ft_max["batch_size"]}, \
        \nlearning_rate={ft_max["learning_rate"]}, \
        \neval_f1={ft_max["eval_f1"]}')

Run: 0 / 20


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,1.147003,0.725490,0.823899
2,0.232700,1.539302,0.710784,0.802676
3,0.113000,1.699150,0.715686,0.801370
4,0.079200,1.790454,0.708333,0.797963


Run: 1 / 20


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,2.366890,0.710784,0.815047
2,0.016900,2.378123,0.710784,0.799320
3,0.009300,2.246092,0.732843,0.819237
4,0.010300,2.298892,0.727941,0.814691


Run: 2 / 20


...

Run: 18 / 20


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,4.613530,0.691176,0.785714
2,No log,4.747046,0.671569,0.764085
3,No log,4.927131,0.661765,0.751799
4,No log,4.920716,0.661765,0.751799


Run: 19 / 20


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,4.820378,0.676471,0.766784
2,No log,4.781433,0.676471,0.770035
3,No log,4.796065,0.676471,0.770035
4,No log,4.810355,0.674020,0.767888



Result 0:         
batch_size=8,         
learning_rate=0.0003,         
eval_f1=0.797962648556876

Result 1:         
batch_size=8,         
learning_rate=0.0001,         
eval_f1=0.8146911519198664

Result 2:         
batch_size=8,         
learning_rate=5e-05,         
eval_f1=0.8093645484949833

Result 3:         
batch_size=8,         
learning_rate=3e-05,         
eval_f1=0.8118811881188119

Result 4:         
batch_size=16,         
learning_rate=0.0003,         
eval_f1=0.7918781725888325

Result 5:         
batch_size=16,         
learning_rate=0.0001,         
eval_f1=0.7877758913412564

Result 6:         
batch_size=16,         
learning_rate=5e-05,         
eval_f1=0.7827586206896552

Result 7:         
batch_size=16,         
learning_rate=3e-05,         
eval_f1=0.7848537005163512

Result 8:         
batch_size=32,         
learning_rate=0.0003,         
eval_f1=0.791095890410959

Result 9:         
batch_size=32,         
learning_rate=0.0001,         
eval_f1=0.7938671

In [54]:
import csv

with open('ft-results.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=list(ft_results[0].keys()))

    writer.writeheader()
    for result in ft_results:
      writer.writerow(result)

# LoRA

In [2]:
class LoraModule(nn.Module):
  def __init__(self, in_features, out_features, rank, alpha):
    super().__init__()
    self.scale = alpha / rank
    self.A = nn.Parameter(torch.randn(in_features, rank))
    self.B = nn.Parameter(torch.zeros(rank, out_features))

  def forward(self, x):
    return (self.scale * (x @ self.A @ self.B))

class LoraLinear(nn.Module):
  def __init__(self, linear, rank, alpha):
    super().__init__()
    if (isinstance(linear, LoraLinear)):
      self.linear = linear.linear
      self.lora = LoraModule(self.linear.in_features, self.linear.out_features, rank, alpha)

    else:
      self.linear = linear
      self.lora = LoraModule(self.linear.in_features, self.linear.out_features, rank, alpha)

  def forward(self, x):
    return self.linear(x) + self.lora(x)

In [3]:
def get_trainable_parameters(model):
  trainable_parameters = 0
  parameters = 0
  for param in model.parameters():
    parameters += param.numel()
    if param.requires_grad:
      trainable_parameters += param.numel()
  return parameters, trainable_parameters

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [16]:
def configure_lora_model(model, rank, alpha):
  for i, layer in enumerate(model.bert.encoder.layer):
    s = layer.attention.self
    s.query = LoraLinear(s.query, rank, alpha)
    s.value = LoraLinear(s.value, rank, alpha)

  for name, param in model.named_parameters():
    if 'A' in name or 'B' in name:
      param.requires_grad = True
    else:
      param.requires_grad = False
  params, trainable_params = get_trainable_parameters(model)
  print(f"Total Parameters={params}, Trainable Parameters={trainable_params}")

  return (params, trainable_params)

In [17]:
ranks = [1, 2, 4, 8, 16]
alphas = [1, 2, 4, 8, 16]

In [19]:
from transformers import Trainer, TrainingArguments, set_seed

set_seed(0)
results = []
runs = len(batch_sizes)*len(learning_rates)*len(ranks)*len(alphas)
run = 0
for rank in ranks:
  for alpha in alphas:
    for size in batch_sizes:
      for rate in learning_rates:
        run += 1
        print(f"Run: {run}/{runs}")
        result = {"batch_size": size, "learning_rate": rate, "rank": rank, "alpha": alpha}

        params, trainable_params = configure_lora_model(model, rank, alpha)
        result["parameters"] = params
        result["trainable_parameters"] = trainable_params

        training_args = TrainingArguments(
            "test-trainer",
            eval_strategy="epoch",
            per_device_eval_batch_size=size,
            per_device_train_batch_size=size,
            num_train_epochs=epochs,
            learning_rate=rate,
            disable_tqdm=True,
            report_to="none"
        )

        trainer = Trainer(
            model,
            training_args,
            train_dataset=tokenized_datasets["train"],
            eval_dataset=tokenized_datasets["validation"],
            data_collator=data_collator,
            processing_class=tokenizer,
            compute_metrics=compute_metrics,
        )

        result = result | trainer.train().metrics | trainer.evaluate()
        result.append(result)

max = results[0]
for i, result in enumerate(results):
  if result['eval_f1'] > max['eval_f1']:
    max = result

print(f'\n\n================\
        \nBest Result: \
        \nrank={max["rank"]}, \
        \nalpha={max["alpha"]}, \
        \nparameters={max["parameters"]}, \
        \ntrainable_parameters={max["trainable_parameters"]}, \
        \nbatch_size={max["batch_size"]}, \
        \nlearning_rate={max["learning_rate"]}, \
        \neval_f1={max["eval_f1"]}')

Total Parameters=4387202, Trainable Parameters=1024


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,4.456339,0.708333,0.799325
2,0.000200,4.497383,0.710784,0.803333
3,0.000000,4.474108,0.710784,0.803333
4,0.002500,4.495666,0.708333,0.801336


Total Parameters=4387202, Trainable Parameters=1024


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,4.475583,0.713235,0.804020
2,0.000300,4.478076,0.710784,0.802676
3,0.000300,4.468445,0.713235,0.804020
4,0.003600,4.470316,0.713235,0.804020


Total Parameters=4387202, Trainable Parameters=1024


...

Total Parameters=4402562, Trainable Parameters=16384


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,4.508591,0.713235,0.804020
2,No log,4.543034,0.713235,0.804020
3,No log,4.558310,0.713235,0.804020
4,No log,4.566176,0.713235,0.804020


Total Parameters=4402562, Trainable Parameters=16384


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,4.495546,0.713235,0.804020
2,No log,4.518611,0.713235,0.804020
3,No log,4.530028,0.713235,0.804020
4,No log,4.536084,0.713235,0.804020



\Result 0:         
rank=1,         
alpha=1,         
parameters=4387202,         
trainable_parameters=1024,         
batch_size=8,         
learning_rate=0.0003,         
training=TrainOutput(global_step=1836, training_loss=0.0014122390991770754, metrics={'train_runtime': 24.9435, 'train_samples_per_second': 588.209, 'train_steps_per_second': 73.606, 'total_flos': 2616141208032.0, 'train_loss': 0.0014122390991770754, 'epoch': 4.0}),         
evaluation={'eval_loss': 4.495665550231934, 'eval_accuracy': 0.7083333333333334, 'eval_f1': 0.8013355592654424, 'eval_runtime': 1.4783, 'eval_samples_per_second': 275.99, 'eval_steps_per_second': 34.499, 'epoch': 4.0}

\Result 1:         
rank=1,         
alpha=1,         
parameters=4387202,         
trainable_parameters=1024,         
batch_size=8,         
learning_rate=0.0001,         
training=TrainOutput(global_step=1836, training_loss=0.0016132970228953558, metrics={'train_runtime': 24.882, 'train_samples_per_second': 589.664, 'train_ste

In [46]:
import csv

with open('lora-results.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=list(results[0].keys()))

    writer.writeheader()
    for result in results:
      writer.writerow(result)